In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap 
from mpl_toolkits.basemap import Basemap, addcyclic
from my_tools import file_dic, plot_dic
from glob import glob
from scipy.stats import ttest_ind

%matplotlib qt

In [2]:
path = '/media/onno/Algemeen/Thesis/'
path_to_write = '/media/onno/Algemeen/Thesis/fcst_composite/hovmoller/reanalysis/'

In [3]:
#load RWP property files
rean_env_gen = xr.open_dataset(path +file_dic['envelope']['ERA5']).sel(latitude=slice(90,0)).squeeze()
rean_v300_gen = xr.open_dataset(path + file_dic['v_wind']['ERA5']).sel(latitude=slice(90,0)).squeeze()
rean_env_gen =  rean_env_gen.assign_coords(longitude=(((rean_env_gen.longitude + 180) % 360) - 180)).sortby('longitude')
rean_v300_gen =  rean_v300_gen.assign_coords(longitude=(((rean_v300_gen.longitude + 180) % 360) - 180)).sortby('longitude')

rean_env_gen=rean_env_gen.rename({'longitude':'lon'});rean_env_gen=rean_env_gen.rename({'latitude':'lat'})
rean_v300_gen=rean_v300_gen.rename({'longitude':'lon'});rean_v300_gen=rean_v300_gen.rename({'latitude':'lat'})
# rean_env_gen = rean_env_gen.sel(time=[bool(i) for i in rean_env_gen.time.dt.hour%24==0])
# rean_v300_gen = rean_v300_gen.sel(time=[bool(i) for i in rean_v300_gen.time.dt.hour%24==0])

ERA5RF_env_gen = xr.open_dataset(path+file_dic['envelope']['ERA5RF'],decode_times=False).squeeze()
GFS_env_gen = xr.open_dataset(path+file_dic['envelope']['GFS'],decode_times=False).squeeze()
# ERA5RF_v300_gen = xr.open_dataset(path+'era5rf_v300_wledit2000-10000_latavg_0-240h_12hourly_2x2nh_jan79-dec19.nc',decode_times=False).squeeze()
# GFS_v300_gen = xr.open_dataset(path+'gefsrf2_v300_control0-252h_6hourly_2x2_dec84-nov19.nc',decode_times=False).squeeze()

GFS_init_time = pd.Timestamp('1800-01-01')
ERA5RF_init_time = pd.Timestamp('1900-01-01')
GFS_env_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_env_gen.time.values]
# GFS_v300_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_v300_gen.time.values]
ERA5RF_env_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_env_gen.time.values]
# ERA5RF_v300_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_v300_gen.time.values]

# GFS_env_gen = GFS_env_gen.rename({'lon':'longitude','lat':'latitude'}); GFS_v300_gen = GFS_v300_gen.rename({'lon':'longitude','lat':'latitude'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'lon':'longitude','lat':'latitude'}); ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'lon':'longitude','lat':'latitude'})
GFS_env_gen = GFS_env_gen.assign_coords(lon=(((GFS_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_v300_gen = GFS_v300_gen.assign_coords(lon=(((GFS_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
ERA5RF_env_gen = ERA5RF_env_gen.assign_coords(lon=(((ERA5RF_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# ERA5RF_v300_gen = ERA5RF_v300_gen.assign_coords(lon=(((ERA5RF_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_env_gen = GFS_env_gen.sel(lead=GFS_env_gen.lead.values[[bool(i) for i in GFS_env_gen.lead%24==0]])
# GFS_v300_gen = GFS_v300_gen.sel(lead=GFS_v300_gen.lead.values[[bool(i) for i in GFS_v300_gen.lead%24==0]])
# ERA5RF_env_gen = ERA5RF_env_gen.sel(lead=ERA5RF_env_gen.lead.values[[bool(i) for i in ERA5RF_env_gen.lead%24==0]])
# # ERA5RF_v300_gen = ERA5RF_v300_gen.sel(lead=ERA5RF_v300_gen.lead.values[[bool(i) for i in ERA5RF_v300_gen.lead%24==0]])

# GFS_env_gen['lead'] = GFS_env_gen.lead.values//24
# GFS_v300_gen['lead'] = GFS_v300_gen.lead.values//24
# ERA5RF_env_gen['lead'] = ERA5RF_env_gen.lead.values//24
# # ERA5RF_v300_gen['lead'] = ERA5RF_v300_gen.lead.values//24

# GFS_env_gen = GFS_env_gen.rename({'latitude':'lat','longitude':'lon'})
# GFS_v300_gen = GFS_v300_gen.rename({'latitude':'lat','longitude':'lon'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'latitude':'lat','longitude':'lon'})
# # ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'latitude':'lat','longitude':'lon'})





Hovmöller Diagram for RWP Envelope and V300 at any location and temperature event for best 10% of the forecasts

In [29]:
coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine
file = '/GFS_T850/rank_forecast/{}_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}_10percent.txt'
eventz = ['persistent_hw','persistent_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
                     'persistent_cw': np.arange(24,61,2),
                     'short_hw':np.arange(24,61,2),
                     'short_cw':np.arange(24,61,2)},
               'JJA':{'persistent_hw':np.arange(16,61,2),
                     'persistent_cw': np.arange(16,61,2),
                     'short_hw':np.arange(16,61,2),
                     'short_cw':np.arange(16,61,2)}}

lead_day = 5

    
#         datez_event = np.load(path + 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
for model in modelz:
    for lat_0,lat_1,lon_0,lon_1 in coordinatez:
        reanz_env_good = []
        reanz_v300_good = []
        reanz_env_bad = []
        reanz_v300_bad = []
        for i,event in enumerate(eventz):
            for season in seasonz:
                df_good = pd.read_csv(path+file.format('good',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_bad = pd.read_csv(path+file.format('bad',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_good.index = pd.to_datetime(df_good.index)
                df_bad.index = pd.to_datetime(df_bad.index)

                lat_max = lat_0 + 8
                lat_min = lat_1 - 10
                for date in df_good.index:
                    time_begin = date - pd.Timedelta(10,'days')
                    time_end = date + pd.Timedelta(10,'days')
                    rean_env_good_sub = rean_env_gen.sel(lat=slice(lat_max,lat_min),
                                                   time=slice(time_begin,time_end))
                    rean_v300_good_sub = rean_v300_gen.sel(lat=slice(lat_max,lat_min),
                                                   time=slice(time_begin,time_end))
                    rean_env_good_sub = rean_env_good_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_good_sub.time)))
                    rean_v300_good_sub = rean_v300_good_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_good_sub.time)))
                    rean_env_good_sub = rean_env_good_sub.assign_coords(date=date).expand_dims('date')
                    rean_v300_good_sub = rean_v300_good_sub.assign_coords(date=date).expand_dims('date')
                    reanz_env_good.append(rean_env_good_sub)
                    reanz_v300_good.append(rean_v300_good_sub)
#                 rean_env_good = xr.concat(reanz_env_good,dim='time').groupby('time').mean().mean('lat')
#                 rean_v300_good = xr.concat(reanz_v300_good,dim='time').groupby('time').mean().mean('lat')
                for date in df_bad.index:
                    time_begin = date - pd.Timedelta(10,'days')
                    time_end = date + pd.Timedelta(10,'days')
                    rean_env_bad_sub = rean_env_gen.sel(lat=slice(lat_max,lat_min),
                                                   time=slice(time_begin,time_end))
                    rean_v300_bad_sub = rean_v300_gen.sel(lat=slice(lat_max,lat_min),
                                                    time=slice(time_begin,time_end))
                    rean_env_bad_sub = rean_env_bad_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_bad_sub.time)))
                    rean_v300_bad_sub = rean_v300_bad_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_bad_sub.time)))
                    rean_env_bad_sub = rean_env_bad_sub.assign_coords(date=date).expand_dims('date')
                    rean_v300_bad_sub = rean_v300_bad_sub.assign_coords(date=date).expand_dims('date')
                    reanz_env_bad.append(rean_env_bad_sub)
                    reanz_v300_bad.append(rean_v300_bad_sub)
#                 rean_env_bad = xr.concat(reanz_env_bad,dim='time').groupby('time').mean().mean('lat')
#                 rean_v300_bad = xr.concat(reanz_v300_bad,dim='time').groupby('time').mean().mean('lat')

        rean_env_good = xr.concat(reanz_env_good,dim='date').mean('lat')
        rean_v300_good = xr.concat(reanz_v300_good,dim='date').mean('lat')
        rean_env_bad = xr.concat(reanz_env_bad,dim='date').mean('lat')
        rean_v300_bad = xr.concat(reanz_v300_bad,dim='date').mean('lat')
        rean_env_good.attrs['lon_0']=lon_0
        rean_v300_good.attrs['lon_0']=lon_0
        rean_env_bad.attrs['lon_0']=lon_0
        rean_v300_bad.attrs['lon_0']=lon_0
        rean_env_good.to_netcdf(path_to_write+'/rawdata/rean_env_good_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_20percent.nc'\
                                .format(lon_0,lon_1,lat_0,lat_1,lead_day,model))
        rean_v300_good.to_netcdf(path_to_write+'/rawdata/rean_v300_good_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_20percent.nc'\
                                .format(lon_0,lon_1,lat_0,lat_1,lead_day,model))
        rean_env_bad.to_netcdf(path_to_write+'/rawdata/rean_env_bad_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_20percent.nc'\
                                .format(lon_0,lon_1,lat_0,lat_1,lead_day,model))
        rean_v300_bad.to_netcdf(path_to_write+'/rawdata/rean_v300_bad_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_20percent.nc'\
                                .format(lon_0,lon_1,lat_0,lat_1,lead_day,model))








Combined Hovmöllers (for all regions for every type of event)

In [ ]:
plt.rcParams.update({'hatch.color': 'k'})

coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine

modelz = ['GFS','ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
lead_day = 5
letterz_ax1 = ['a','b']
letterz_ax2 = ['c','d']
file_good = '/rawdata/rean_env_good_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_20percent.nc'
file_bad = '/rawdata/rean_env_bad_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_20percent.nc'

fig,axz = plt.subplots(2,2,figsize=(16,9))
for i,model in enumerate(modelz):
    reanz_env_good_total = []
    reanz_env_bad_total = []
    for lat_0,lat_1,lon_0,lon_1 in coordinatez[::-1]:
        rean_env_good = xr.open_dataset(path_to_write+file_good.format(lon_0,lon_1,lat_0,lat_1,lead_day,model))
        rean_env_good = rean_env_good.assign_coords(lon=(((rean_env_good.lon-lon_0+180)%360)-180)).sortby('lon')
        rean_env_bad = xr.open_dataset(path_to_write+file_bad.format(lon_0,lon_1,lat_0,lat_1,lead_day,model))
        rean_env_bad = rean_env_bad.assign_coords(lon=(((rean_env_bad.lon-lon_0+180)%360)-180)).sortby('lon')
        reanz_env_good_total.append(rean_env_good)
        reanz_env_bad_total.append(rean_env_bad)
    rean_env_good_total = xr.concat(reanz_env_good_total,dim='date')
    rean_env_bad_total = xr.concat(reanz_env_bad_total,dim='date')
    rean_env_good_total.to_netcdf(path_to_write+'rawdata/rean_env_good_ALL_SEPERATE_{}_20percent.nc'.format(model))
    rean_env_bad_total.to_netcdf(path_to_write+'rawdata/rean_env_bad_ALL_SEPERATE_{}_20percent.nc'.format(model))
    assert(len(rean_env_good_total.date)==len(rean_env_bad_total.date))
    
    rean_env_good_mean = rean_env_good_total.mean('date')
    rean_env_bad_mean = rean_env_bad_total.mean('date')
    p_value_num = ttest_ind(rean_env_good_total.v.values,rean_env_bad_total.v.values,axis=0)[1]
    p_value_bool = p_value_num<0.1
    lonz = rean_env_good_mean.lon.values
    dayz = rean_env_good_mean.time.values
    clr_levels_contourf = np.arange(10,25.01,.1)
    clr_levels_contour = np.arange(10,25.01,1)
    ax1 = axz[0,i]
    im=ax1.contourf(lonz,dayz,rean_env_good_mean.v,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    ax1.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
    cs=ax1.contour(lonz,dayz,rean_env_good_mean.v,colors='k',levels=clr_levels_contour,extend='both')
    ax1.clabel(cs,inline=True,fmt='%.0f')
    ax1.set_title(model_labelz[i] + '\n Good Forecasts',fontsize=15)
    ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
    ax1.axhline(y=0,color='grey')
    ax1.set_ylabel('Days before/after onset event',fontsize=12)
    ax1.set_xlabel('Pseudo Longitude [°]',fontsize=12)
    ax1.set_xticks([-120,-60,0,60,120])
    ax1.set_xticklabels([])
    ax1.tick_params(labelsize=12)
    ax1.set_yticks([-10,-5,0,5,10])
    ax1.text(x=0,y=1.1,s=letterz_ax1[i]+')',transform=ax1.transAxes,fontweight='bold',fontsize=17,
                   verticalalignment='top')

    ax2 = axz[1,i]
    ax2.contourf(lonz,dayz,rean_env_bad_mean.v,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    ax2.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
    cs=ax2.contour(lonz,dayz,rean_env_bad_mean.v,colors='k',levels=clr_levels_contour,extend='both')
    ax2.clabel(cs,inline=True,fmt='%.0f')
    ax2.set_title('Bad Forecasts',fontsize=15)
    ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
    ax2.axhline(y=0,color='grey')
    ax2.set_ylabel('Days before/after onset event',fontsize=12)
    ax2.set_xlabel('Pseudo Longitude [°]',fontsize=12)
    ax2.set_xticks([-120,-60,0,60,120])
    ax2.set_yticks([-10,-5,0,5,10])
    ax2.tick_params(labelsize=12)
    ax2.text(x=0,y=1.1,s=letterz_ax2[i]+')',transform=ax2.transAxes,fontweight='bold',fontsize=17,
                   verticalalignment='top')

    ax1.label_outer(); ax2.label_outer()
cbar_ax = fig.add_axes([0.2, 0.07, 0.6, 0.02])
cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
cbar.ax.get_xaxis().set_ticks([10,15,20,25])
cbar.ax.set_xticklabels([10,15,20,25])
cbar.ax.set_xlabel('E (m/s)',fontsize=12)
cbar.ax.tick_params(labelsize=12)
fig.subplots_adjust(left=0.05,right=0.97,bottom=0.16,top=0.88,hspace=0.15,wspace=0.05)
fig.suptitle("Averaged Hovmöller Good and Bad Forecasts on Day 5 Forecast : N = {}".format(len(rean_env_good_total.date)),
            fontsize=20)
        
    

pasteCombined Hovmöllers for each region individual)

In [82]:
plt.rcParams.update({'hatch.color': 'k'})

coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine

modelz = ['GFS','ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
lead_day = 5
letterz_ax1 = ['a','b']
letterz_ax2 = ['c','d']
file_good = '/rawdata/rean_env_good_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_33percent.nc'
file_bad = '/rawdata/rean_env_bad_lon_{}_{}_lat_{}_{}_lead_day_{}_{}_33percent.nc'


for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    fig,axz = plt.subplots(2,2,figsize=(16,9))
    for i,model in enumerate(modelz):
        rean_env_good = xr.open_dataset(path_to_write+file_good.format(lon_0,lon_1,lat_0,lat_1,lead_day,model)).load()
        rean_env_good = rean_env_good.assign_coords(lon=(((rean_env_good.lon-lon_0+180)%360)-180)).sortby('lon')
        rean_env_bad = xr.open_dataset(path_to_write+file_bad.format(lon_0,lon_1,lat_0,lat_1,lead_day,model)).load()
        rean_env_bad = rean_env_bad.assign_coords(lon=(((rean_env_bad.lon-lon_0+180)%360)-180)).sortby('lon')
        print(file_good.format(lon_0,lon_1,lat_0,lat_1,lead_day,model))
        print(file_bad.format(lon_0,lon_1,lat_0,lat_1,lead_day,model))
#         print(float(rean_env_good.v[0,0,0]))
#         print(float(rean_env_bad.v[0,0,0]))
#         reanz_env_good_total.append(rean_env_good)
#         reanz_env_bad_total.append(rean_env_bad)
#     rean_env_good_total = xr.concat(reanz_env_good_total,dim='date')
#     rean_env_bad_total = xr.concat(reanz_env_bad_total,dim='date')
#     rean_env_good_total.to_netcdf(path_to_write+'rawdata/rean_env_good_ALL_SEPERATE_{}.nc'.format(model))
#     rean_env_bad_total.to_netcdf(path_to_write+'rawdata/rean_env_bad_ALL_SEPERATE_{}.nc'.format(model))
        assert(len(rean_env_good.date)==len(rean_env_bad.date))
        rean_env_good_mean = rean_env_good.mean('date')
        rean_env_bad_mean = rean_env_bad.mean('date')
        p_value_num = ttest_ind(rean_env_good.v.values,rean_env_bad.v.values,axis=0)[1]
        p_value_bool = p_value_num<0.1
        lonz = rean_env_good_mean.lon.values
        dayz = rean_env_good_mean.time.values
        clr_levels_contourf = np.arange(10,25.01,.1)
        clr_levels_contour = np.arange(10,25.01,1)
        ax1 = axz[0,i]
        print(float(rean_env_good_mean.v[0,0]))
        im=ax1.contourf(lonz,dayz,rean_env_good_mean.v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
        ax1.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax1.contour(lonz,dayz,rean_env_good_mean.v.values,colors='k',levels=clr_levels_contour,extend='both')
        ax1.clabel(cs,inline=True,fmt='%.0f')
        ax1.set_title(model_labelz[i] + '\n Good Forecasts',fontsize=15)
        ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
        ax1.axhline(y=0,color='grey')
        ax1.set_ylabel('Days before/after onset event',fontsize=12)
        ax1.set_xlabel('Pseudo Longitude [°]',fontsize=12)
        ax1.set_xticks([-120,-60,0,60,120])
        ax1.set_xticklabels([])
        ax1.tick_params(labelsize=12)
        ax1.set_yticks([-10,-5,0,5,10])
        ax1.text(x=0,y=1.1,s=letterz_ax1[i]+')',transform=ax1.transAxes,fontweight='bold',fontsize=17,
                       verticalalignment='top')

        ax2 = axz[1,i]
        
        ax2.contourf(lonz,dayz,rean_env_bad_mean.v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
        ax2.contourf(lonz,dayz,p_value_bool,3,hatches=[None,'xx'],colors='none')
        cs=ax2.contour(lonz,dayz,rean_env_bad_mean.v.values,colors='k',levels=clr_levels_contour,extend='both')
        ax2.clabel(cs,inline=True,fmt='%.0f')
        ax2.set_title('Bad Forecasts',fontsize=15)
        ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
        ax2.axhline(y=0,color='grey')
        ax2.set_ylabel('Days before/after onset event',fontsize=12)
        ax2.set_xlabel('Pseudo Longitude [°]',fontsize=12)
        ax2.set_xticks([-120,-60,0,60,120])
        ax2.set_yticks([-10,-5,0,5,10])
        ax2.tick_params(labelsize=12)
        ax2.text(x=0,y=1.1,s=letterz_ax2[i]+')',transform=ax2.transAxes,fontweight='bold',fontsize=17,
                       verticalalignment='top')

        ax1.label_outer(); ax2.label_outer()
    cbar_ax = fig.add_axes([0.2, 0.07, 0.6, 0.02])
    cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
    cbar.ax.get_xaxis().set_ticks([10,15,20,25])
    cbar.ax.set_xticklabels([10,15,20,25])
    cbar.ax.set_xlabel('E (m/s)',fontsize=12)
    cbar.ax.tick_params(labelsize=12)
    fig.subplots_adjust(left=0.05,right=0.97,bottom=0.16,top=0.88,hspace=0.15,wspace=0.05)
    fig.suptitle("{}-{} °N {}-{} °E Day {} Forecast: N = {}".format(lat_1,lat_0,lon_0,lon_1,lead_day,
                                                                   len(rean_env_good.date)),fontsize=20)
    fig.savefig(path_to_write+'/figures/hovmoeller_envelope_lon_{}_{}_lat_{}_{}'.format(lon_0,lon_1,lat_0,lat_1))
    plt.close(fig)
    

/rawdata/rean_env_good_lon_0_8_lat_62_54_lead_day_5_GFS_33percent.nc
/rawdata/rean_env_bad_lon_0_8_lat_62_54_lead_day_5_GFS_33percent.nc
17.377910614013672
/rawdata/rean_env_good_lon_0_8_lat_62_54_lead_day_5_ERA5RF_33percent.nc
/rawdata/rean_env_bad_lon_0_8_lat_62_54_lead_day_5_ERA5RF_33percent.nc
18.047258377075195
/rawdata/rean_env_good_lon_0_8_lat_54_46_lead_day_5_GFS_33percent.nc
/rawdata/rean_env_bad_lon_0_8_lat_54_46_lead_day_5_GFS_33percent.nc
17.677371978759766
/rawdata/rean_env_good_lon_0_8_lat_54_46_lead_day_5_ERA5RF_33percent.nc
/rawdata/rean_env_bad_lon_0_8_lat_54_46_lead_day_5_ERA5RF_33percent.nc
18.61768913269043
/rawdata/rean_env_good_lon_0_8_lat_46_38_lead_day_5_GFS_33percent.nc
/rawdata/rean_env_bad_lon_0_8_lat_46_38_lead_day_5_GFS_33percent.nc
16.008464813232422
/rawdata/rean_env_good_lon_0_8_lat_46_38_lead_day_5_ERA5RF_33percent.nc
/rawdata/rean_env_bad_lon_0_8_lat_46_38_lead_day_5_ERA5RF_33percent.nc
17.129302978515625
/rawdata/rean_env_good_lon_8_16_lat_62_54_lead_

Combined Hovmöllers for all persistent extremes (envelope only)

In [4]:
coordinatez = [
(54,46,6,14),#Germany
# (44,36,352,360), #Spain
# (54,46,26,34), #Ukraine
(58,50,352,360), #UK
# (42,34,28,36), #Turkey
# (68,60,22,30), #Finland
(66,58,6,14)] #Norway/Sweden
# (60,52,46,54)] #Russia Samara/Kazan region
path = '/media/onno/Algemeen/Thesis/fcst_composite/hovmoller/reanalysis/'
file = 'rawdata/{}_forecast_{}_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}.nc'
eventz = ['persistent_hw','persistent_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(20,35,1),
                     'persistent_cw': np.arange(20,35,2),
                     'short_hw':np.arange(20,35,1),
                     'short_cw':np.arange(20,35,1)},
               'JJA':{'persistent_hw':np.arange(15,25,1),
                     'persistent_cw': np.arange(15,25,1),
                     'short_hw':np.arange(15,25,1),
                     'short_cw':np.arange(15,25,1)}}
lead_day = 5


ds_good_event_modelz = []
ds_bad_event_modelz = []
for model in modelz:
    ds_good_eventz = []
    ds_bad_eventz = []
    for i,event in enumerate(eventz):
        ds_good_seasonz = []
        ds_bad_seasonz = []
        for season in seasonz:
            ds_good_regionz = []
            ds_bad_regionz = []
            for lat_0,lat_1,lon_0,lon_1 in coordinatez:
                file_good_env = file.format('good','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                file_bad_env = file.format('bad','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                ds_good = xr.open_dataset(path+file_good_env)
                ds_bad = xr.open_dataset(path+file_bad_env)
                lon_0 = ds_good.attrs['lon_0']
                ds_good = ds_good.assign_coords(lon=(((ds_good.lon-lon_0+180)%360)-180)).sortby('lon')
                ds_bad = ds_bad.assign_coords(lon=(((ds_bad.lon-lon_0+180)%360)-180)).sortby('lon')
                ds_good_regionz.append(ds_good)
                ds_bad_regionz.append(ds_bad)
            ds_good_region = xr.concat(ds_good_regionz,dim='region').mean(dim='region')
            ds_bad_region = xr.concat(ds_bad_regionz,dim='region').mean(dim='region')
            ds_good_seasonz.append(ds_good_region)
            ds_bad_seasonz.append(ds_bad_region)
        ds_good_season = xr.concat(ds_good_seasonz,dim='season').mean(dim='season')
        ds_bad_season = xr.concat(ds_bad_seasonz,dim='season').mean(dim='season')
        ds_good_eventz.append(ds_good_season)
        ds_bad_eventz.append(ds_bad_season)
    ds_good_event = xr.concat(ds_good_eventz,dim='event').mean(dim='event')
    ds_bad_event = xr.concat(ds_bad_eventz,dim='event').mean(dim='event')
    ds_good_event.to_netcdf(path+'rawdata/good_forecast_envelope_hovmoller_persistent_NW_EUROPE_COMBINED_{}.nc'.format(model))
    ds_good_event.to_netcdf(path+'rawdata/bad_forecast_envelope_hovmoller_persistent_NW_EUROPE_COMBINED_{}.nc'.format(model))
    ds_good_event_modelz.append(ds_good_event)
    ds_bad_event_modelz.append(ds_bad_event)




fig,axz=plt.subplots(2,2,figsize=(16,9))
for i in np.arange(len(modelz)):
    ax1 =axz[0,i];ax2=axz[1,i]
    lonz = ds_good_event.lon.values
    dayz = ds_good_event.time.values

    clr_levels_contourf = np.arange(10,25.01,.1)
    clr_levels_contour = np.arange(10,25.01)
    im = ax1.contourf(lonz,dayz,ds_good_event_modelz[i].v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    cs = ax1.contour(lonz,dayz,ds_good_event_modelz[i].v.values,colors='k',levels=clr_levels_contour,extend='both')
    ax1.clabel(cs,inline=True,fmt='%.0f')

    ax1.set_title(model_labelz[i] + '\n Good Forecasts')
    ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
    ax1.axhline(y=0,color='grey')
    ax1.set_ylabel('Days before/after onset event')
    ax1.set_xlabel('Pseudo Longitude [°]')


    ax2.contourf(lonz,dayz,ds_bad_event_modelz[i].v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    cs = ax2.contour(lonz,dayz,ds_bad_event_modelz[i].v.values,colors='k',levels=clr_levels_contour,extend='both')
    ax2.clabel(cs,inline=True,fmt='%.0f')

    ax2.set_title('Bad Forecasts')
    ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
    ax2.axhline(y=0,color='grey')
    ax2.set_ylabel('Days before/after onset event')
    ax2.set_xlabel('Pseudo Longitude [°]')
    fig.suptitle('RWP Envelope Composite Analysis All Persistent Extremes')

    fig.subplots_adjust(bottom=0.2,left=0.05,right=0.95,wspace=0.05,hspace=0.15)
    cbar_ax = fig.add_axes([0.1, 0.08, 0.8, 0.045])
    cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
    cbar.ax.get_xaxis().set_ticks([10,15,20,25])
    cbar.ax.set_xticklabels([10,15,20,25])
    cbar.ax.set_xlabel('E (m/s)',fontsize=12)
    ax1.label_outer(); ax2.label_outer()
sys.exit()
fig.savefig(path+'combined/persistent_extremes_NW_EUROPE.png')

#                 fig.savefig(path+'good_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_alt.png'.format(event,season,model))
#                 plt.close(fig)

/home/onno/miniconda3/envs/thesis/lib/python3.6/site-packages/ipykernel_launcher.py:100: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


SystemExit: 

/home/onno/miniconda3/envs/thesis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [66]:
coordinatez = [
(54,46,6,14),#Germany
# (44,36,352,360), #Spain
# (54,46,26,34), #Ukraine
(58,50,352,360), #UK
# (42,34,28,36), #Turkey
# (68,60,22,30), #Finland
(66,58,6,14)] #Norway/Sweden
# (60,52,46,54)] #Russia Samara/Kazan region
path = '/media/onno/Algemeen/Thesis/fcst_composite/hovmoller/reanalysis/'
file = 'rawdata/{}_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_v2.nc'
eventz = ['persistent_hw','persistent_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(20,35,1),
                     'persistent_cw': np.arange(20,35,2),
                     'short_hw':np.arange(20,35,1),
                     'short_cw':np.arange(20,35,1)},
               'JJA':{'persistent_hw':np.arange(15,25,1),
                     'persistent_cw': np.arange(15,25,1),
                     'short_hw':np.arange(15,25,1),
                     'short_cw':np.arange(15,25,1)}}
lead_day = 5


ds_good_event_modelz = []
ds_bad_event_modelz = []
for model in modelz:
    ds_good_eventz = []
    ds_bad_eventz = []
    for i,event in enumerate(eventz):
        ds_good_seasonz = []
        ds_bad_seasonz = []
        for season in seasonz:
            ds_env_good = xr.open_dataset(path+file.format('good',event,season,model))
            ds_env_bad = xr.open_dataset(path+file.format('bad',event,season,model))
            ds_good_seasonz.append(ds_env_good)
            ds_bad_seasonz.append(ds_env_bad)
        ds_good_season = xr.concat(ds_good_seasonz,dim='season').mean(dim='season')
        ds_bad_season = xr.concat(ds_bad_seasonz,dim='season').mean(dim='season')
        ds_good_eventz.append(ds_good_season)
        ds_bad_eventz.append(ds_bad_season)
    ds_good_event = xr.concat(ds_good_eventz,dim='event').mean(dim='event')
    ds_bad_event = xr.concat(ds_bad_eventz,dim='event').mean(dim='event')
    ds_good_event.to_netcdf(path+'rawdata/good_forecast_envelope_hovmoller_persistent_NW_EUROPE_COMBINED_{}_v2.nc'.format(model))
    ds_good_event.to_netcdf(path+'rawdata/bad_forecast_envelope_hovmoller_persistent_NW_EUROPE_COMBINED_{}_v2.nc'.format(model))
    ds_good_event_modelz.append(ds_good_event)
    ds_bad_event_modelz.append(ds_bad_event)



fig,axz=plt.subplots(2,2,figsize=(16,9))
for i in np.arange(len(modelz)):
    ax1 =axz[0,i];ax2=axz[1,i]
    lonz = ds_good_event.lon.values
    dayz = ds_good_event.time.values

    clr_levels_contourf = np.arange(10,25.01,.1)
    clr_levels_contour = np.arange(10,25.01)
    im = ax1.contourf(lonz,dayz,ds_good_event_modelz[i].v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    cs = ax1.contour(lonz,dayz,ds_good_event_modelz[i].v.values,colors='k',levels=clr_levels_contour,extend='both')
    ax1.clabel(cs,inline=True,fmt='%.0f')

    ax1.set_title(model_labelz[i] + '\n Good Forecasts')
    ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
    ax1.axhline(y=0,color='grey')
    ax1.set_ylabel('Days before/after onset event')
    ax1.set_xlabel('Pseudo Longitude [°]')


    ax2.contourf(lonz,dayz,ds_bad_event_modelz[i].v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    cs = ax2.contour(lonz,dayz,ds_bad_event_modelz[i].v.values,colors='k',levels=clr_levels_contour,extend='both')
    ax2.clabel(cs,inline=True,fmt='%.0f')

    ax2.set_title('Bad Forecasts')
    ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
    ax2.axhline(y=0,color='grey')
    ax2.set_ylabel('Days before/after onset event')
    ax2.set_xlabel('Pseudo Longitude [°]')
    fig.suptitle('RWP Envelope Composite Analysis All Persistent Extremes')

    fig.subplots_adjust(bottom=0.2,left=0.05,right=0.95,wspace=0.05,hspace=0.15)
    cbar_ax = fig.add_axes([0.1, 0.08, 0.8, 0.045])
    cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
    cbar.ax.get_xaxis().set_ticks([10,15,20,25])
    cbar.ax.set_xticklabels([10,15,20,25])
    cbar.ax.set_xlabel('E (m/s)')
    ax1.label_outer(); ax2.label_outer()
sys.exit()
fig.savefig(path+'combined/persistent_extremes_NW_EUROPE.png')

#                 fig.savefig(path+'good_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_alt.png'.format(event,season,model))
#                 plt.close(fig)

/home/onno/miniconda3/envs/thesis/lib/python3.6/site-packages/ipykernel_launcher.py:88: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [ ]:
coordinatez = [
(54,46,6,14),#Germany
(44,36,352,360), #Spain
(54,46,26,34), #Ukraine
(58,50,352,360), #UK
(42,34,28,36), #Turkey
(68,60,22,30), #Finland
(66,58,6,14), #Norway/Sweden
(60,52,46,54), #Russia Samara/Kazan region #Germany
]

region_labelz = ['DE','ES','UA','UK','TR','FI','NO','RU']
path = '/media/onno/Algemeen/Thesis/fcst_composite/hovmoller/reanalysis/'
file = 'rawdata/{}_forecast_{}_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}.nc'
eventz = ['persistent_hw','persistent_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['DJF']
modelz = ['GFS','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(20,35,1),
                     'persistent_cw': np.arange(20,35,2),
                     'short_hw':np.arange(20,35,1),
                     'short_cw':np.arange(20,35,1)},
               'JJA':{'persistent_hw':np.arange(15,25,1),
                     'persistent_cw': np.arange(15,25,1),
                     'short_hw':np.arange(15,25,1),
                     'short_cw':np.arange(15,25,1)}}
lead_day = 5

for model in modelz:
    count = 0
    for lat_0,lat_1,lon_0,lon_1 in coordinatez:
        ds_good_eventz = []
        ds_bad_eventz = []
        for i,event in enumerate(eventz):
            ds_good_seasonz = []
            ds_bad_seasonz = []
            for season in seasonz:

                file_good_env = file.format('good','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                file_bad_env = file.format('bad','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                ds_good = xr.open_dataset(path+file_good_env)
                ds_bad = xr.open_dataset(path+file_bad_env)
                ds_good_seasonz.append(ds_good)
                ds_bad_seasonz.append(ds_bad)
            ds_good_season = xr.concat(ds_good_seasonz,dim='season').mean(dim='season')
            ds_bad_season = xr.concat(ds_bad_seasonz,dim='season').mean(dim='season')
            ds_good_eventz.append(ds_good_season)
            ds_bad_eventz.append(ds_bad_season)
        ds_good_event = xr.concat(ds_good_eventz,dim='event').mean(dim='event')
        ds_bad_event = xr.concat(ds_bad_eventz,dim='event').mean(dim='event')
        ds_good_event.to_netcdf(path+'rawdata/good_forecast_envelope_hovmoller_persistent_lon_{}_{}_lat_{}_{}_{}_{}.nc'.format(
        lon_0,lon_1,lat_0,lat_1,model,season))
        ds_good_event.to_netcdf(path+'rawdata/bad_forecast_envelope_hovmoller_persistent_lon_{}_{}_lat_{}_{}_{}_{}.nc'.format(
        lon_0,lon_1,lat_0,lat_1,model,season))






        fig,axz=plt.subplots(2,1,figsize=(16,9))
        ax1 =axz.flat[0];ax2=axz.flat[1]
        lonz = ds_good_event.lon.values
        dayz = ds_good_event.time.values

        clr_levels_contourf = np.arange(10,35.01,.1)
        clr_levels_contour = np.arange(10,35.01,2)
        im = ax1.contourf(lonz,dayz,ds_good_event.v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
        cs = ax1.contour(lonz,dayz,ds_good_event.v.values,colors='k',levels=clr_levels_contour,extend='both')
        ax1.clabel(cs,inline=True,fmt='%.0f')

        ax1.set_title('Good Forecasts')
        ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
        ax1.axhline(y=0,color='grey')
        ax1.set_ylabel('Days before/after onset event')


        ax2.contourf(lonz,dayz,ds_bad_event.v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
        cs = ax2.contour(lonz,dayz,ds_bad_event.v.values,colors='k',levels=clr_levels_contour,extend='both')
        ax2.clabel(cs,inline=True,fmt='%.0f')

        ax2.set_title('Bad Forecasts')
        ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
        ax2.axhline(y=0,color='grey')
        ax2.set_ylabel('Days before/after onset event')
        ax2.set_xlabel('Longitude [°]')
        fig.suptitle('RWP Envelope Composite Analysis All Persistent Extremes {} {} {}'.format(region_labelz[count],model,season))

        fig.subplots_adjust(right=0.8)
        cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
        cbar = fig.colorbar(im, cax=cbar_ax)
        cbar.ax.get_yaxis().set_ticks([10,15,20,25])
        cbar.ax.set_yticklabels([10,15,20,25])
        cbar.ax.set_ylabel('v (m/s)')
        fig.savefig(path+'combined/persistent_extremes_lon_{}_{}_lat_{}_{}_{}_{}.png'.format(lon_0,lon_1,lat_0,lat_1,model,season))
        plt.close(fig)
        count += 1

#                 fig.savefig(path+'good_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_alt.png'.format(event,season,model))
#                 plt.close(fig)